Nota gee_env for now

to to: make an env file for you

In [1]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests

from sentinelhub import (SHConfig, DataCollection, SentinelHubCatalog, SentinelHubRequest, BBox, bbox_to_dimensions, CRS, MimeType, Geometry)



c:\Users\trygg\miniconda3\envs\sentinelHub_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# # Import credentials
# from creds import *

def get_keycloak(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
        }
    try:
        r = requests.post("https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
        data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Keycloak token creation failed. Reponse from the server was: {r.json()}"
            )
    return r.json()["access_token"]
        

username = 'tryggvims@hotmail.com'
password = 'Mamamia.854288'

keycloak_token = get_keycloak(username, password)

In [3]:
start_date = "2021-06-01"
end_date = "2021-06-10"
data_collection = "SENTINEL-1"
aoi = "POLYGON((4.220581 50.958859,4.521264 50.953236,4.545977 50.906064,4.541858 50.802029,4.489685 50.763825,4.23843 50.767734,4.192435 50.806369,4.189689 50.907363,4.220581 50.958859))'"

In [9]:
jsonx = requests.get(f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq '{data_collection}' and OData.CSC.Intersects(area=geography'SRID=4326;{aoi}) and ContentDate/Start gt {start_date}T00:00:00.000Z and ContentDate/Start lt {end_date}T00:00:00.000Z").json()
json = pd.DataFrame.from_dict(jsonx['value']).head(10)

In [11]:
import pandas as pd

# Assuming your DataFrame is named 'json'
# Replace 'json' with the actual name of your DataFrame

# Sort DataFrame by 'OriginDate' column
json_sorted = json.sort_values(by='OriginDate')

# Display every line in the sorted DataFrame
pd.set_option('display.max_rows', None)  # Set to display all rows
print(json_sorted)

# Reset display options if needed
# pd.reset_option('display.max_rows')


    @odata.mediaContentType                                    Id  \
0  application/octet-stream  da5616a3-bc8f-5ea8-a6b4-ea42c2f81c48   
4  application/octet-stream  01ad16b9-ada2-5a02-9f82-c162bd4673d6   
2  application/octet-stream  15b32cb7-a88c-5f03-868e-a69ef5743034   
1  application/octet-stream  1e470af2-8213-527c-8a34-8f59f16af06f   
9  application/octet-stream  52c2bdfe-0fb9-5ddb-ac7d-4c9054668bad   
7  application/octet-stream  3c8fcfdf-4131-5919-ab89-152a11084d5a   
5  application/octet-stream  e406f705-9a3e-5fc4-b952-c468795be575   
6  application/octet-stream  1caf133b-99ad-5024-9138-43718321d7fd   
3  application/octet-stream  63be5b46-51ca-448a-998b-efe292034088   
8  application/octet-stream  f0ad2be3-6f28-4b8d-8d7d-050ba7bc131c   

                                                Name  \
0  S1A_20210605T173256__S1B_20210530T173205_20210...   
4  S1B_20210608T055802__S1A_20210602T055832_20210...   
2  S1A_IW_GRDH_1SDV_20210605T173257_20210605T1733...   
1  S1A_IW_SLC__1

In [ ]:
session = requests.Session()
session.headers.update({'Authorization': f'Bearer {keycloak_token}'})
url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products(6f77dc3e-c918-5b42-911a-e8213eab8929)/$value"
response = session.get(url, allow_redirects=False)
while response.status_code in (301, 302, 303, 307):
    url = response.headers['Location']
    response = session.get(url, allow_redirects=False)

file = session.get(url, verify=False, allow_redirects=True)

with open(f"outputs/test.zip", 'wb') as p:
    p.write(file.content)

### Downloading SLCs

 1e470af2-8213-527c-8a34-8f59f16af06f er SLC. Það tók 114 mín að hala því niður

In [40]:
### nýtt download dæmi

import requests
id_product = "6f77dc3e-c918-5b42-911a-e8213eab8929"
access_token = keycloak_token

url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products(1e470af2-8213-527c-8a34-8f59f16af06f)/$value"

headers = {"Authorization": f"Bearer {access_token}"}

session = requests.Session()
session.headers.update(headers)
response = session.get(url, headers=headers, stream=True)

with open("outputs/product_newdownloadtestSLC.zip", "wb") as file:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            file.write(chunk)

In [ ]:
json = requests.get("https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name,'S1') and ContentDate/Start gt 2022-05-03T00:00:00.000Z and ContentDate/Start lt 2022-05-03T00:10:00.000Z&$orderby=ContentDate/Start&$top=100").json()
df = pd.DataFrame.from_dict(json['value'])

# Print only specific columns
columns_to_print = ['Id', 'Name','S3Path','GeoFootprint']  
df[columns_to_print].head(3)

In [ ]:
json = requests.get("https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq 'SENTINEL-1' and ContentDate/Start gt 2022-05-03T00:00:00.000Z and ContentDate/Start lt 2023-05-03T00:11:00.000Z").json()
df = pd.DataFrame.from_dict(json['value'])

# Print only specific columns
columns_to_print = ['Id', 'Name','S3Path','GeoFootprint']  
df[columns_to_print].head(10)